# P4 (Містобудування, ЖКГ, побутове, торговельне обслуговування, транспорт і зв’язок): 5/6

In [1]:
import numpy as np
import pandas as pd
from src.utils import normalize_parameter, save_data
from src.utils import PATH_RAW, PATH_INTERIM

---

## p4_02
### наявність генеральних планів у містах та селищах міського типу
## p4_03
### наявність генеральних планів у селах
*Тут слід звести з попереднім датасетом*

In [2]:
df_P04_005 = pd.read_excel(PATH_INTERIM / "P4" / "P04_005.xlsx")
df_P04_005["param_type"] = np.select(
    condlist=[
        df_P04_005["Тип населеного пункту (село, селище міського типу, місто)"].isin(["місто", "смт"]),
        df_P04_005["Тип населеного пункту (село, селище міського типу, місто)"].isin(["село", "селище"])
    ],
    choicelist=["p4_02_raw", "p4_03_raw"],
    default=np.nan
)
df_P04_005["plan"] = np.where(
    df_P04_005["Наявність затвердженого Генерального плану населеного пункту (так/ні)"].eq("так"), 
    1,
    0
)

df_P04_005_fin = df_P04_005.groupby(["region", "param_type"], as_index=False)["plan"].mean()
df_P04_005_fin = df_P04_005_fin.pivot(index='region', columns='param_type', values='plan').reset_index()

df_P04_005_fin["p4_02"] = normalize_parameter(df_P04_005_fin["p4_02_raw"], min_bound=0, max_bound=1)
df_P04_005_fin["p4_03"] = normalize_parameter(df_P04_005_fin["p4_03_raw"], min_bound=0, max_bound=1)

feature_range=(0, 1); fill_na=True; array_bounds=(0.30985915492957744, 1.0); normalization_bounds=(0, 1), reverse=False
feature_range=(0, 1); fill_na=True; array_bounds=(0.0, 1.0); normalization_bounds=(0, 1), reverse=False



## p4_05
### протяжність автомобільних доріг загального користування місцевого значення, на яких проведено капітальний ремонт, реконструкцію, відносно загальної довжини доріг

In [3]:
# roads = pd.read_csv(PATH_RAW / "P99" / "all_roads.csv").rename(columns={"Область": "region"})
# df_P04_007 = pd.read_excel(PATH_RAW / "P4" / "P04_007.xlsx")

# df_P04_007 = df_P04_007.loc[pd.to_datetime(df_P04_007["Дата початку ремонтних робіт"]) < "2020-10-01"]

# df_P04_007_fin = pd.concat([
#     roads.groupby("region")["Довжина (протяжність) дороги (км)"].sum().rename('roads_all'),
#     df_P04_007.groupby('region').agg(
#         price=("Загальна вартість ремонту (грн)", "sum"),
#         roads_repaired=("Довжина (протяжність) відремонтованої ділянки (км)", "sum")
#     )
# ], axis=1)

# df_P04_007_fin['price_km'] = df_P04_007_fin['price']/df_P04_007_fin['roads_repaired']
# df_P04_007_fin['p4_05_raw'] = df_P04_007_fin['roads_repaired']/df_P04_007_fin['roads_all']
# df_P04_007_fin = df_P04_007_fin.reset_index().rename(columns={"index": "region"})

# df_P04_007_fin.loc[:, df_P04_007_fin.columns.str.contains("region|_raw")].to_excel(PATH_INTERIM / "P4" / "P04_007.xlsx", index=False)

In [4]:
df_P04_007_fin = pd.read_excel(PATH_INTERIM / "P4" / "P04_007.xlsx")

df_P04_007_fin["p4_05"] = normalize_parameter(df_P04_007_fin["p4_05_raw"], min_bound=0)

feature_range=(0, 1); fill_na=True; array_bounds=(0.0, 0.1640336420980848); normalization_bounds=(0, 0.1640336420980848), reverse=False


## p4_06
### рівень використання коштів дорожнього фонду, передбачених для будівництва, реконструкції, ремонту та утримання автомобільних доріг загального користування місцевого значення, вулиць і доріг комунальної власності у населених пунктах

## p4_07
### середній рівень будівельної готовності об'єктів Великого будівництва (за касовими видатками)
## p4_08
### середній рівень освоєння коштів, виділених на об'єкти Великого будівництва з усіх джерел

In [5]:
df_P04_009 = pd.read_excel(PATH_INTERIM / "P4" / "P04_009.xlsx").rename(columns={"Готовність за касовими видатками": "p4_07_raw"})
df_P04_009["p4_07"] = normalize_parameter(df_P04_009["p4_07_raw"].div(100), min_bound=0, max_bound=1)
df_P04_009["p4_08"] = normalize_parameter(df_P04_009["p4_08_raw"].div(100), min_bound=0, max_bound=1)

feature_range=(0, 1); fill_na=True; array_bounds=(0.2722727272727273, 0.746); normalization_bounds=(0, 1), reverse=False
feature_range=(0, 1); fill_na=True; array_bounds=(0.34538249749666833, 0.9601208010074157); normalization_bounds=(0, 1), reverse=False


## Група 1: p4_02-p4_03, генплани
## Група 2: p4_05-p4_06, дороги
## Група 3: p4_07-p4_08, велике будівництво

In [6]:
dict_weights = {
    'p4_02':1,
    'p4_03':1,
    'p4_05':1,
    'p4_06':1,
    'p4_07':1,
    'p4_08':1
}

sources = [
    df_P04_005_fin, 
    df_P04_007_fin,
#     df_KMU,
    df_P04_009
]
save_data(sources, dict_weights, 'P4', show_results=True)

,region,p4_02,p4_03,p4_05,p4_07,p4_08,P4
0,Вінницька,0.957447,0.798216,0.082180,0.418000,0.788864,5.074511
1,Волинська,1.000000,0.878932,0.065116,0.484974,0.584087,5.021849
2,Дніпропетровська,0.742424,0.543554,0.136325,0.446429,0.792756,4.435813
3,Донецька,0.595506,0.445604,0.000000,0.602128,0.567976,3.685355
4,Житомирська,0.309859,0.304648,0.091628,0.635409,0.646074,3.312698
5,Закарпатська,1.000000,0.929188,0.000000,0.558741,0.471550,4.932465
6,Запорізька,0.873418,0.587356,0.064932,0.404000,0.783673,4.522299
7,Івано-Франківська,1.000000,1.000000,0.000000,0.583333,0.476198,5.099219
8,Київська,0.963636,0.827709,0.000000,0.492749,0.760371,5.074109
9,Кіровоградська,0.878788,0.692553,0.178974,0.272273,0.478321,4.168181
